# Name(s)
**Ethan Moore and Ajay Patel**

**Instructions:** Pair programming assignment. Submit only a single notebook, but make sure to include your first and last names.

# Bayesian Classifier

## Preface
(Courtesy of Dr. Alex Dekhtyar)

The core objective of Knowledge Discovery in Data/Data Mining/Machine Learning methods is to provide efficient algorithms for gaining insight from data. CSC 466 primarily studies the methods and the algorithms that enable
such insight, and that specifically take this insight above and beyond traditional statistical analysis of data (more
about this — later in the course).
However, the true power of KDD/DM/ML methods that we will study in this course is witnessed only when
these methods are applied to actually gain insight from the data. As such, in this course, the deliverables for your
laboratory assignments will be partitioned into two categories:

1. KDD Method implementation. In most labs you will be asked to implement from scratch one or more
KDD method for producing a special type of insight from data. This part of the labs is similar to your other
CS coursework - you will submit your code, and, sometimes, your tests and/or output.

2. Insight, a.k.a., data analysis. For each lab assignment we will provide one or more datasets for your
perusal, and will ask you to perform the analysis of these datasets using the methods you implemented. The
results of this analysis, i.e., the insight, are as important for successful completion of your assignments, as
your implementations. Most of the time, you will be asked to submit a lab report detailing your analysis,
and containing the answers to the questions you are asked to study.
The insight portion of your deliverables is something that you may be seeing for the first time in your CS
coursework. It is not an afterthought in your lab assignments. Your grade will, in no small part, depend on
the results of your analysis, and the writing quality on your report. This lab assignment, and further assignments
will include detailed insturctions on how to prepare reports, and we will discuss report writing several times as
the course progresses.

## Lab Assignment

This is a pair programming assignment. I strongly
discourage individual work for this (and other team/pair programming) lab(s), even if you think you can do it
all by yourself. Also, this is a pair programming assignment, not a ”work in teams of two” assignment. Pair
programming requires joint work on all aspects of the project without delegating portions of the work to individual
1
team members. For this lab, I want all your work — discussion, software development, analysis of the results,
report writing — to be products of joint work.
Students enrolled in the class can pair with other students enrolled in the class. Students on the waitlist can
pair with other students on the waitlists. In the cases of ”odd person out” situations, a team of three people can
be formed, but that team must (a) ask and answer one additional question, and (b) work as a pair would, without
delegation of any work off-line.

For this lab, we are going to first implement a empirical naive bayesian classifier, then implement a feature importance measure and apply it to a dataset, and finally, we will examine the affect of modifying the priors.

For developing this lab, we can use the Titanic Kaggle dataset.

In [1]:
import pandas as pd
titanic_df = pd.read_csv(
    "https://raw.githubusercontent.com/dlsun/data-science-book/master/data/titanic.csv"
)
titanic_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


We only need a few columns, and I will also perform some preprocessing for you:

In [2]:
features = ['pclass','survived','sex','age']
titanic_df = titanic_df.loc[:,features]
display(titanic_df)
titanic_df.loc[:,'pclass']=titanic_df['pclass'].fillna(titanic_df['pclass'].mode()).astype(int)
titanic_df.loc[:,'age']=titanic_df['age'].fillna(titanic_df['age'].median())
titanic_df.loc[:,'age']=(titanic_df['age']/10).astype(str).str[0].astype(int)*10
titanic_df

,pclass,survived,sex,age
0,1,1,female,29.0000
1,1,1,male,0.9167
2,1,0,female,2.0000
3,1,0,male,30.0000
4,1,0,female,25.0000
...,...,...,...,...
1304,3,0,female,14.5000
1305,3,0,female,NaN
1306,3,0,male,26.5000
1307,3,0,male,27.0000


,pclass,survived,sex,age
0,1,1,female,20
1,1,1,male,0
2,1,0,female,0
3,1,0,male,30
4,1,0,female,20
...,...,...,...,...
1304,3,0,female,10
1305,3,0,female,20
1306,3,0,male,20
1307,3,0,male,20


In [3]:
titanic_df.describe()

,pclass,survived,age
count,1309.000000,1309.000000,1309.000000
mean,2.294882,0.381971,24.385027
std,0.837836,0.486055,13.387598
min,1.000000,0.000000,0.000000
25%,2.000000,0.000000,20.000000
50%,3.000000,0.000000,20.000000
75%,3.000000,1.000000,30.000000
max,3.000000,1.000000,80.000000


## Exercise 0
In your own words, describe the preprocessing steps I took above.

You selected only the four columns, (pclass, age, sex, survived), as the four features to create the Bayesian Model. You replaced missing pclass values with the mode of the pclass column. Similarly, you filled the missing values in the age column with the median age and lastly, you rounded down all the ages to the nearest 10. Example being 29 --> 20. 

## Exercise 1
Fill in the following function to determine the prior probability of the classes. The result must be in the form of a Python dictionary such as ``priors = {0: 0.4, 1: 0.6}``.
<pre>
def compute_priors(y):
  ???
  return priors
</pre>

In [4]:
def compute_priors(y, yname="y"):
    priors = y.value_counts(normalize=True)
    return priors.to_dict()

compute_priors(titanic_df['survived'])

{0: 0.6180290297937356, 1: 0.3819709702062643}

## Exercise 2
The next function to implement is the specific class conditional probability:
<pre>
def specific_class_conditional(y,yv,x,xv):
  ???
  
  P(x=xv|y=yv)
  
  return prob
</pre>

In [5]:
def specific_class_conditional(y,yv,x,xv):
    
#     prob = len(titanic_df[(x == xv) & (y == yv)])/len(titanic_df[y == yv])
    prob = ((y == yv) & (x==xv)).sum() / (y==yv).sum()
    return prob


specific_class_conditional(titanic_df['survived'],0,titanic_df['sex'],'female')

0.15698393077873918

## Exercise 3
Now construct a dictionary based data structure that stores all possible class conditional probabilities (e.g., loop through all possible combinations of values). The keys in your dictionary should be of the form "pclass=1|survived=0".

ALL POSSIBLE PCLASSES GIVEN OTHER COLUMN VALUES

<pre>
# X is a dataframe that does not contain the class column y.
def class_conditional(X,y):
  ???
  return probs
  
def prior(y):
  ???
  return probs
</pre>

In [6]:
titanic_df.drop("survived",axis=1)[titanic_df.drop("survived",axis=1).pclass == 1]

,pclass,sex,age
0,1,female,20
1,1,male,0
2,1,female,0
3,1,male,30
4,1,female,20
...,...,...,...
318,1,male,20
319,1,female,30
320,1,male,20
321,1,male,60


In [7]:
a = "pclass"
titanic_df[a].unique()

array([1, 2, 3])

In [8]:
def class_conditional(X,y,yname="y"):
    dictionary = {}
    for a in X.columns:
        for b in X[a].unique():
            for c in y.unique():
               
                prob = ((X[a] == b) & (y == c)).sum() / (y == c).sum()
                string = str(a) + '=' + str(b) + '|' + 'survived=' + str(c)
 
                dictionary[string] = prob
 
    return dictionary

In [9]:
# YOUR SOLUTION HERE

display(class_conditional(titanic_df.drop("survived",axis=1),titanic_df["survived"]))

display(class_conditional(titanic_df.drop("survived",axis=1),titanic_df["survived"],yname="survived"))

display(compute_priors(titanic_df["survived"],yname="survived"))

{'pclass=1|survived=1': 0.4,
 'pclass=1|survived=0': 0.15203955500618047,
 'pclass=2|survived=1': 0.238,
 'pclass=2|survived=0': 0.19530284301606923,
 'pclass=3|survived=1': 0.362,
 'pclass=3|survived=0': 0.6526576019777504,
 'sex=female|survived=1': 0.678,
 'sex=female|survived=0': 0.15698393077873918,
 'sex=male|survived=1': 0.322,
 'sex=male|survived=0': 0.8430160692212608,
 'age=20|survived=1': 0.4,
 'age=20|survived=0': 0.5030902348578492,
 'age=0|survived=1': 0.1,
 'age=0|survived=0': 0.03955500618046971,
 'age=30|survived=1': 0.196,
 'age=30|survived=0': 0.16563658838071693,
 'age=40|survived=1': 0.104,
 'age=40|survived=0': 0.10259579728059333,
 'age=60|survived=1': 0.02,
 'age=60|survived=0': 0.027194066749072928,
 'age=50|survived=1': 0.064,
 'age=50|survived=0': 0.04697156983930779,
 'age=70|survived=1': 0.002,
 'age=70|survived=0': 0.007416563658838072,
 'age=10|survived=1': 0.112,
 'age=10|survived=0': 0.10754017305315204,
 'age=80|survived=1': 0.002,
 'age=80|survived=0':

{'pclass=1|survived=1': 0.4,
 'pclass=1|survived=0': 0.15203955500618047,
 'pclass=2|survived=1': 0.238,
 'pclass=2|survived=0': 0.19530284301606923,
 'pclass=3|survived=1': 0.362,
 'pclass=3|survived=0': 0.6526576019777504,
 'sex=female|survived=1': 0.678,
 'sex=female|survived=0': 0.15698393077873918,
 'sex=male|survived=1': 0.322,
 'sex=male|survived=0': 0.8430160692212608,
 'age=20|survived=1': 0.4,
 'age=20|survived=0': 0.5030902348578492,
 'age=0|survived=1': 0.1,
 'age=0|survived=0': 0.03955500618046971,
 'age=30|survived=1': 0.196,
 'age=30|survived=0': 0.16563658838071693,
 'age=40|survived=1': 0.104,
 'age=40|survived=0': 0.10259579728059333,
 'age=60|survived=1': 0.02,
 'age=60|survived=0': 0.027194066749072928,
 'age=50|survived=1': 0.064,
 'age=50|survived=0': 0.04697156983930779,
 'age=70|survived=1': 0.002,
 'age=70|survived=0': 0.007416563658838072,
 'age=10|survived=1': 0.112,
 'age=10|survived=0': 0.10754017305315204,
 'age=80|survived=1': 0.002,
 'age=80|survived=0':

{0: 0.6180290297937356, 1: 0.3819709702062643}

## Exercise 4
Now you are ready to calculate the posterior probabilities for a given sample. Write and test the following function that returns a dictionary where the keys are of the form "pclass=1,sex=male,age=60|survived=0". Make sure you return 0 if the specific combination of values does not exist.
<pre>
def posteriors(probs,priors,x):
    return probs
</pre>

In [10]:
def posteriors(probs,priors,x):
    dictionary = {}
   
    pclasskey = "pclass=" + str(x.pclass) + "|survived=0"
    #print(pclasskey)
    agekey = "age=" + str(x.age) + "|survived=0"
    sexkey = "sex=" + str(x.sex) + "|survived=0"
   
    answer = (probs[pclasskey] * probs[agekey] * probs[sexkey]  * priors[0]) / (titanic_df.pclass.value_counts(normalize=True)[x.pclass] *
    titanic_df.age.value_counts(normalize=True)[x.age] *
    titanic_df.sex.value_counts(normalize=True)[x.sex])
   
    string =  "surived=0|" + "pclass=" + str(x.pclass) + ",sex=" + str(x.sex) + ",age=" + str(x.age)
    string2 = "surived=1|" + "pclass=" + str(x.pclass) + ",sex=" + str(x.sex) + ",age=" + str(x.age)
   
    dictionary[string] = answer
    dictionary[string2] = 1-answer
   
    return dictionary


In [11]:
probs = class_conditional(titanic_df.drop("survived",axis=1),titanic_df["survived"],yname="survived")
priors = compute_priors(titanic_df["survived"],yname="survived")
posteriors(probs,priors,titanic_df.drop("survived",axis=1).loc[0])

{'surived=0|pclass=1,sex=female,age=20': 0.18218321481929167,
 'surived=1|pclass=1,sex=female,age=20': 0.8178167851807083}

## Exercise 5
All this is great, but how would you evaluate how we are doing? Let's write a function call train_test_split that splits our dataframe into approximately training and testing dataset. Make sure it does this randomly.
<pre>
def train_test_split(X,y,test_frac=0.5):
   return Xtrain,ytrain,Xtest,ytest
</pre>

In [12]:
import random
import numpy
 
def train_test_split(X,y,test_frac=0.5):
    sample = numpy.random.choice(len(X)-1, int(len(X)*test_frac), replace=False)
    Xtrain = X.loc[sample]
    ytrain = y.loc[sample]
    Xtest = X.drop(sample) #X[~X.isin(Xtrain)].dropna()
    ytest = y.drop(sample) #y[~y.isin(ytrain)].dropna()
   
    
    return Xtrain,ytrain,Xtest,ytest
 

In [13]:
train_test_split(titanic_df.drop("survived",axis=1),titanic_df["survived"])
Xtrain,ytrain,Xtest,ytest=train_test_split(titanic_df.drop("survived",axis=1),titanic_df["survived"])
Xtrain,ytrain,Xtest,ytest

(      pclass     sex  age
 900        3    male   20
 744        3    male   10
 1255       3    male   20
 899        3  female   20
 855        3    male   10
 ...      ...     ...  ...
 301        1    male   40
 428        2  female   20
 251        1  female   20
 619        3    male   10
 985        3    male   20
 
 [654 rows x 3 columns], 900     0
 744     0
 1255    0
 899     1
 855     0
        ..
 301     0
 428     1
 251     1
 619     0
 985     1
 Name: survived, Length: 654, dtype: int64,       pclass     sex  age
 0          1  female   20
 4          1  female   20
 5          1    male   40
 10         1    male   40
 11         1  female   10
 ...      ...     ...  ...
 1298       3    male   30
 1300       3  female   10
 1302       3    male   20
 1306       3    male   20
 1308       3    male   20
 
 [655 rows x 3 columns], 0       1
 4       0
 5       1
 10      0
 11      1
        ..
 1298    0
 1300    1
 1302    0
 1306    0
 1308    0
 Name: survived

## Exercise 6
For this exercise, find the conditional probabilities and the priors using a training dataset of size 70% and then using these probabilities find the accuracy if they are used to predict the test dataset. 

In [14]:
def test_model():
    predictions = []
    Xtrain,ytrain,Xtest,ytest=train_test_split(titanic_df.drop("survived",axis=1),titanic_df["survived"], test_frac = 0.7)
   
    probs = class_conditional(Xtrain,ytrain)
    priors = compute_priors(ytrain)
 
    for i in Xtest.index:
        value = -1
        dictionary = posteriors(probs,priors,Xtest.loc[i])
        for j in dictionary.keys():
            #print(j)
            k = j.split("|")
            result = k[0].split("=")[1]
            if dictionary[j] > value:
                value = dictionary[j]
                finalresult = result
   
        predictions.append(int(finalresult)) #take this int out if responses are not numeric
   
    Xtest["prediction"] = predictions
   
    return Xtest, ytest
 
Xtest,ytest = test_model()
Xtest

,pclass,sex,age,prediction
1,1,male,0,1
3,1,male,30,1
4,1,female,20,1
10,1,male,40,0
12,1,female,20,1
...,...,...,...,...
1294,3,male,20,0
1295,3,male,20,0
1300,3,female,10,1
1302,3,male,20,0


In [15]:
count = 0
for i in Xtest.index:
    if ytest.loc[i] == Xtest.prediction.loc[i]: 
        count += 1
count / len(Xtest)

0.7862595419847328

## Exercise 7
For this exercise, you must improve/extend your methods above as necessary to compute the accuracy of predicting the activity from the dataset we've generated in class. Once we have filled out this dataset, I will provide a csv file as well as any preprocessing similar to the Titanic. You may have to modify your functions above to work with both datasets or you may not (depending of course on how you wrote them).

In [16]:
import pandas as pd
df = pd.read_csv('activity.csv')
df.head()

,deadline,lazy,party,age,activity
0,Soon,No,Yes,22,Party
1,Soon,Yes,No,20,TV
2,None,No,Yes,21,Bar
3,Urgent,Yes,No,21,Study
4,Soon,Yes,No,20,Study


In [17]:
def class_conditional2(X,y, yname = "survived"):
    dictionary = {}
    for a in X.columns:
        for b in X[a].unique():
            for c in y.unique():
               
                prob = ((X[a] == b) & (y == c)).sum() / (y == c).sum()
                string =  str(yname) + '=' + str(c) + '|' + str(a) + '=' + str(b)
 
                dictionary[string] = prob
 
    return dictionary
 
import pandas as pd
df=pd.read_csv('activity.csv')
df.activity.value_counts()

Study    16
TV       14
Party    11
Bar       4
Name: activity, dtype: int64

In [18]:
def posteriors2(probs,priors,x):
    dictionary = {}
   
    for i in ["Party", "Bar", "TV", "Study"]:
   
        deadlinekey = "activity=" + i + "|deadline=" + str(x.deadline)
        agekey = "activity=" + i + "|age=" + str(x.age)
        lazykey = "activity=" + i + "|lazy=" + str(x.lazy)
        partykey = "activity=" + i + "|party=" + str(x.party)
        
        #print(probs)
       
        #try:
        p1 = probs[agekey]
        p2 = probs[partykey]
        p3 = probs[deadlinekey]
        p4 = probs[lazykey]
        p5 = priors[i]
 
        answer = (p1*p2*p3*p4*p5
                 ) / (df.deadline.value_counts(normalize=True)[x.deadline] *
        df.age.value_counts(normalize=True)[x.age] *
        df.lazy.value_counts(normalize=True)[x.lazy] *
        df.party.value_counts(normalize=True)[x.party])
 
        string =  "activity=" + i + "|" + "deadline=" + str(x.deadline) + ",lazy=" + str(x.lazy) + ",age=" + str(x.age) + ",party=" + str(x.party)
 
        dictionary[string] = answer
 
    return dictionary

In [19]:
def test_model():
    predictions = []
    Xtrain,ytrain,Xtest,ytest=train_test_split(df.drop("activity",axis=1),df["activity"], test_frac = 0.7)
   
    probs = class_conditional2(Xtrain,ytrain, yname = "activity")
    priors = compute_priors(ytrain)
    #print(probs)
 
    for i in Xtest.index:
        value = -1
        dictionary = posteriors2(probs,priors,Xtest.loc[i])
        for j in dictionary.keys():
            k = j.split("|")
            result = k[0].split("=")[1]
            if dictionary[j] > value:
                value = dictionary[j]
                finalresult = result
   
        predictions.append(finalresult)
    
    Xtest["prediction"] = predictions
   
    return Xtest, ytest
 
Xtest,ytest = test_model()
Xtest

,deadline,lazy,party,age,prediction
2,None,No,Yes,21,Party
4,Soon,Yes,No,20,TV
7,Soon,Yes,No,20,TV
12,Soon,Yes,No,21,TV
13,Urgent,Yes,Yes,20,Study
23,Soon,Yes,No,20,TV
27,None,No,Yes,20,Party
29,Soon,Yes,No,20,TV
30,None,Yes,No,20,TV
38,Soon,No,Yes,22,Bar


In [20]:
count = 0
for i in Xtest.index:
    if ytest.loc[i] == Xtest.prediction.loc[i]: #this is because our predictions were predicting whether the person would not survive. Whereas the original df had 1 for survived, ours has 1 for died
        count += 1
count / len(Xtest)

0.5714285714285714

In [21]:
def accuracy_loop(n):
    Xtest,ytest = test_model()
    countx = 0
    for i in range(n):
        count = 0
        for i in Xtest.index:
            if ytest.loc[i] == Xtest.prediction.loc[i]:
                count += 1
        countx += count/len(Xtest)
    return countx/n


In [23]:
OG_Accuracy = accuracy_loop(1000)
OG_Accuracy

0.4285714285714319

## Excercises 8
For this exercise, I would like you to implement the feature importance algorithm describe in [https://christophm.github.io/interpretable-ml-book/feature-importance.html](https://christophm.github.io/interpretable-ml-book/feature-importance.html). After you implement this, what is the most important feature for our in-class activity prediction dataset? Does this feature make sense to you?

In [24]:
def test_model2(X,y, test_frac = 0.7):
    predictions = []
    Xtrain,ytrain,Xtest,ytest=train_test_split(X,y,test_frac)
   
    probs = class_conditional2(Xtrain,ytrain, yname = "activity")
    priors = compute_priors(ytrain)
    #print(probs)
 
    for i in Xtest.index:
        value = -1
        dictionary = posteriors2(probs,priors,Xtest.loc[i])
        for j in dictionary.keys():
            k = j.split("|")
            result = k[0].split("=")[1]
            if dictionary[j] > value:
                value = dictionary[j]
                finalresult = result
   
        predictions.append(finalresult)
    
    Xtest["prediction"] = predictions
   
    return Xtest, ytest

In [25]:
import numpy as np
def permutation(X, y, OG_Accuracy):
   
    Xtest,ytest = test_model2(X,y)
   
#     count = 0
#     for i in Xtest.index:
#         if ytest.loc[i] == Xtest.prediction.loc[i]: 
#             count += 1
#     OG_Accuracy = count / len(Xtest)
   
    Xcopy = X.copy()
   
    for col in Xcopy.columns:
        
        overall_accuracy = 0
        num_iterations = 0
        for i in range(100):
            Xcopy[col] = np.random.permutation(Xcopy[col])
            
            try:
                Xtest2,ytest2 = test_model2(Xcopy,y)

                count = 0
                for i in Xtest2.index:
                    if ytest2.loc[i] == Xtest2.prediction.loc[i]:
                        count += 1
                Accuracy = count / len(Xtest2)

                overall_accuracy += Accuracy
                num_iterations += 1
                Xcopy = X.copy()
            except:
                pass
            
        print("Column Permuted: ", col, " Accuracy:", overall_accuracy/num_iterations, " Original Accuracy:", OG_Accuracy)

In [26]:
permutation(df.drop("activity",axis=1), df["activity"], OG_Accuracy)

Column Permuted:  deadline  Accuracy: 0.4844720496894409  Original Accuracy: 0.4285714285714319
Column Permuted:  lazy  Accuracy: 0.5793650793650795  Original Accuracy: 0.4285714285714319
Column Permuted:  party  Accuracy: 0.4779635258358665  Original Accuracy: 0.4285714285714319
Column Permuted:  age  Accuracy: 0.5266875981161696  Original Accuracy: 0.4285714285714319


Both accuracy calculations are very dependent on how the sample was split due to our small amount of data, so the fact that each permutation re-randomizes the train/test split makes our results not very consistent. It makes the most sense to us given the input data that 'party' would be the highest indicator of activity, which we see corroborated by a fair amount of our feature selection runs.